# running_sherpa_in_notebook_hltau_clean

This notebook runs the series of sherpa scripts used to analyze extracted spectra, using basic scripts for analysis that are stored in basic_scripts.py.

In [1]:
import matplotlib.pyplot as plt
from sherpa.astro.ui import *
import numpy as np
from astropy.table import Table
from astropy import units as u
import matplotlib as mpl
from basic_scripts import *
import time
import pprint
import copy

Load data following an existing script

In [ ]:
%run load_hltau_into_sherpa.py

In [ ]:
# Adjust the analyzed spectral region of the spectra from obsid 0865040401 to account for the flare in XZ Tau during this
# observation.

ignore_id(86504040101,None,1.2)
ignore_id(86504040102,None,1.2)
ignore_id(86504040103,None,1.2)

Find initial best-fit parameters for the data, using a Monte Carlo method to search the full parameter space for a best fit. Free parameters are absorption (a1.nH), plasma temperature (s1.kT), plasma normalization (s1.norm), and abundance of iron relative to solar (s1.Fe). Abundances of calcium, magnesium, nickel, and silicon are fixed to the iron abundance as well. We then use the Levenberg-Marquardt method to re-fit, assuming that the Monte-Carlo method has found the best region of parameter space, for efficiency purposes. Results are stored in a dictionary.

In [ ]:
set_method('moncar')

freeze(s1.Ne)
thaw(s1.kT)

startTime = time.time()
fit(86504050101, 86504050102, 86504050103, 86504030101, 86504030102, 86504030103, 
    86504070101, 86504070102, 86504070103)#, 86504040101, 86504040103)
endTime = time.time()
print(endTime-startTime)

testfit = get_fit_results()
testfitDict = get_results_dict('AllBut401',testfit)

In [ ]:
set_method('levmar')
#fit(50101, 50102, 50103, 20101, 20102, 20103, 30101, 30102, 30103, 70101, 70102, 70103, 60101, 60102, 60103)#, 40101, 40102, 40103)
fit(86504030101, 86504030102, 86504030103, 86504050101, 86504050102, 86504050103, 
    86504070101, 86504070102, 86504070103)
fitresultsOneTOneAbsDict = get_results_dict('FaintXMM',get_fit_results())

In [ ]:
fitresultsOneTOneAbsDict

In [ ]:
#set baseids for obsids, for easier calling going forward.
baseids = [865040201,865040301,865040401,865040501,865040601,865040701]

In [ ]:
#Ensure that we are set properly at our initial model
set_model_from_dict(fitresultsOneTOneAbsDict)

In [ ]:
#Find three-sigma boundaries for our data to ensure that we don't stray too far out of the realm of reality based on our best fits.
set_conf_opt('sigma',3.)
conf(86504050101, 86504050102, 86504050103, 86504030101, 86504030102, 86504030103, 
     86504070101, 86504070102, 86504070103)#, 60101, 60102, 60103)
confresultsFaintXMMbounds = get_conf_results()
confresultsFaintXMMbounds

In [ ]:
print(confresultsFaintXMMbounds)

In [ ]:
#Store bounds as a dictionary
confresultsFaintXMMboundsDict = get_confresults_dict(fitresultsOneTOneAbsDict,confresultsFaintXMMbounds)
confresultsFaintXMMboundsDict

In [ ]:
#Reset our confidence to 1-sigma, find best fits with uncertainties.
set_conf_opt('sigma',1.)
conf(86504050101, 86504050102, 86504050103, 86504030101, 86504030102, 86504030103, 
     86504070101, 86504070102, 86504070103)#, 60101, 60102, 60103)
confresultsFaintXMM = get_conf_results()
confresultsFaintXMM

In [ ]:
confresultsFaintXMMDict = get_confresults_dict(fitresultsOneTOneAbsDict,confresultsFaintXMM)
confresultsFaintXMMDict

In [ ]:
# Start a table to store our results
tableFitResultsFreeFe = Table(rows=[confresultsFaintXMMDict])
tableFitResultsFreeFe

In [ ]:
# Store full list of observation ids to be fit.
baseidsFull = [865040201,865040301,865040401,865040501,865040601,865040701,
               20160,20161,21946,21947,21948,21950,21951,21952,21953,21954,21965,
               20906,18915,
               200810201,200810301,200810401,200810501,200810601,200810701,200810801,200810901,200811001,
               200811101,200811201,
               109060301]

In [ ]:
# Get best fits for all obsids, with various flags applied
tableFitResultsFreeFeFilled = fit_all_obsids_hltau(baseidsFull,tableFitResultsFreeFe,fitresultsOneTOneAbsDict,
                                                   '_FreeFeOneTOneAbs_20240806.png',confresultsFaintXMMboundsDict,5,
                                                   inputbinflag=True)

In [ ]:
tableFitResultsFreeFeFilled.show_in_notebook()

In [ ]:
a1.nH.min = 0.
a1.nH.max = 1000000.0

Use get_flux_info from basic_scripts to fill in table with flux information.

In [ ]:
tableFitResultsFreeFeFilledWithFluxes = get_flux_info(tableFitResultsFreeFeFilled)

In [ ]:
tableFitResultsFreeFeFilledWithFluxes.show_in_notebook()

Do the same things again, but this time with the abundance of iron (and the elements pinned to the iron abundance) frozen at the best-fit value from the joint fit.

In [ ]:
set_model_from_dict(fitresultsOneTOneAbsDict)
freeze(s1.Fe)
fit(86504030101, 86504030102, 86504030103, 86504050101, 86504050102, 86504050103, 
    86504070101, 86504070102, 86504070103)
fitresultsOneTOneAbsFittedFeDict = get_results_dict('FaintXMM',get_fit_results())

conf(86504050101, 86504050102, 86504050103, 86504030101, 86504030102, 86504030103, 
     86504070101, 86504070102, 86504070103)#, 60101, 60102, 60103)
confresultsFaintXMMFittedFe = get_conf_results()

confresultsFaintXMMFittedFeDict = get_confresults_dict(fitresultsOneTOneAbsFittedFeDict,confresultsFaintXMMFittedFe)
confresultsFaintXMMFittedFeDict['obsid'] = 'FaintXMM'
confresultsFaintXMMFittedFeDict

In [ ]:
tableFitResultsFittedFe = Table(rows=[confresultsFaintXMMFittedFeDict])
tableFitResultsFittedFe

In [ ]:
tableFitResultsFittedFeFilled = fit_all_obsids_hltau(baseidsFull,tableFitResultsFittedFe,fitresultsOneTOneAbsFittedFeDict,
                                                     '_FittedFeOneTOneAbs_20240214.png',confresultsFaintXMMboundsDict,5,
                                                     inputbinflag=True)

In [ ]:
a1.nH.min = 0.
a1.nH.max = 1000000.0
tableFitResultsFittedFeFilled.show_in_notebook()

In [ ]:
tableFitResultsFittedFeFilledWithFluxes = get_flux_info(tableFitResultsFittedFeFilled)
tableFitResultsFittedFeFilledWithFluxes.show_in_notebook()

Write tables to file

In [ ]:
tableFitResultsFreeFeFilledWithFluxes.write('hltau_tables/tableFitResultsFreeFe_20240214.ecsv',delimiter=',',overwrite=True)
tableFitResultsFittedFeFilledWithFluxes.write('hltau_tables/tableFitResultsFittedFe_20240214.ecsv',delimiter=',',overwrite=True)